In [1]:
import requests
import xml.etree.ElementTree as ET

In [31]:
pages = range(0, 1013)
years = range(1925, 1948)
months = range(1,13)
month= 6

In [33]:
data = {}
for year in years:
#     for month in months:
    ids = []
    year_month_start = "{0}-{1:02d}".format(year, month)
    print(year_month_start)
    year_month_end = "{0}-{1:02d}".format(year, month+1)
    for page in pages:
        url ="http://api.larm.fm/v6/View/Get?view=Search&sort=PubStartDate%2Bdesc&filter=%28PubStartDate:[{}-01T00:00:00Z%20TO%20{}-30T00:00:00Z]%20AND%20%28Type%3ASchedule%20OR%20Type%3AScheduleNote%29%29&pageIndex={}&pageSize=20&sessionGUID=049da351-b81f-424e-82c4-1162926d3688&format=json&userHTTPStatusCodes=False".format(year_month_start, year_month_end,page)
        r = requests.get(url)
        if r.json()["ModuleResults"][0]["Count"] is 0:
            break
        else:
            ids_ = [i['Id'] for i in  r.json()["ModuleResults"][0]["Results"]]
            for id in ids_:
                ids.append(id)
    data[year_month_start] = ids

1925-06
1926-06
1927-06
1928-06
1929-06
1930-06
1931-06
1932-06
1933-06
1934-06
1935-06
1936-06
1937-06
1938-06
1939-06
1940-06
1941-06
1942-06
1943-06
1944-06
1945-06
1946-06
1947-06


In [34]:
query = "http://api.larm.fm/v6/View/Get?view=Object&query={}&pageIndex=0&pageSize=20&sessionGUID=049da351-b81f-424e-82c4-1162926d3688&format=json&userHTTPStatusCodes=False".format(id)

In [35]:
data_text = {}
for year_month in data:
    if len(data[year_month]):
        print(year_month)
        texts = [] 
        for id in ids:
            query = "http://api.larm.fm/v6/View/Get?view=Object&query={}&pageIndex=0&pageSize=20&sessionGUID=049da351-b81f-424e-82c4-1162926d3688&format=json&userHTTPStatusCodes=False".format(id)
            r = requests.get(query)
            metaxlm = r.json()['ModuleResults'][0]["Results"][0]["Metadatas"][0]["MetadataXml"]
            tree = ET.fromstring(metaxlm)
            texts.append(tree.getchildren()[2].text)
    data_text[year_month] = texts

1926-06
1933-06
1930-06
1928-06
1942-06
1941-06
1925-06
1938-06
1935-06
1937-06
1945-06
1932-06
1936-06
1943-06
1947-06
1934-06
1940-06
1931-06
1939-06
1946-06
1927-06
1944-06
1929-06


In [49]:
data_text_splitted = {}
for ym in data_text:
    data_ym = []
    for data in data_text[ym]:
        words = data.split(" ")
        data_ym.append(words)
    data_text_splitted[ym] =data_ym

In [47]:
import json

In [51]:
with open("results.json", "w") as f:
    json.dump(data_text_splitted, f)

In [46]:
data_text_splitted

{'1925-06': ['Statsradiofonien,\nRosenørns',
  'Alle',
  '22,',
  'v.',
  'r\nep#',
  'NB#\nTILFØJELSER\ntil\nRad',
  'ioprogrammet.\nONSDAG-',
  'DEN',
  '50.',
  'JULI',
  '1947«\n8.00-8,o5.',
  'PRESSENS',
  'RADIOAVIS.\n8,48-8,52.',
  'DAGENS',
  'PROGRAM.\n12.00-14,oo.',
  'MIDDAGSKONCERT',
  'fra',
  "WIVEX.\n,7IV,X'",
  'STRYGi:IENS',
  'EMBLE.',
  'Dirigent:',
  'RICHARD',
  'JOHANSEN.\nJ.Eixners',
  'Spansk',
  'March.\nErl.Frederiksen*',
  'Lad',
  'Donerne',
  'fortælle.\nSmile',
  'aid',
  'teuf',
  'el:',
  'Sirenenzauber•',
  'Vals.\nHugo',
  '&XKXXGyldmark:',
  'Ave',
  'Maria.\nFritz',
  'Kreisler:',
  'V',
  '»',
  'ienermarch.\nS.Yradier:',
  'La',
  'Paloma#\nKai',
  'Mortensen:',
  'Den',
  'drilagtige',
  'Violin.\nH.C.Lumbye:',
  'Krolis',
  'Balklange.\nEric',
  'Christiansen:',
  'La',
  'Passione.\nv/Emil',
  'Reesen:',
  'Fantasi',
  'over',
  'Melodier',
  'af',
  'Edvard',
  'Grieg,\nGade:',
  'Efteraar.\nAmilcare',
  'Ponchielli:',
  'Timernes',
  'Dans',
 